In [1]:
# !pip install resnest
# !pip install pretrainedmodels
# !pip install geffnet
# !pip install apex
# !pip install -user albumentations 
# !pip install git+https://github.com/ildoonet/pytorch-gradual-warmup-lr.git

In [1]:
import torch
import torch.nn as nn
import geffnet
from resnest.torch import resnest101
from pretrainedmodels import se_resnext101_32x4d

import os
import time
import random
import argparse
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data.sampler import RandomSampler, SequentialSampler
from torch.optim.lr_scheduler import CosineAnnealingLR

import apex
from apex import amp

C:\Users\sky31\anaconda3\lib\site-packages\apex-0.1-py3.8.egg\apex\pyprof\__init__.py:5: FutureWarning: pyprof will be removed by the end of June, 2022
  warnings.warn("pyprof will be removed by the end of June, 2022", FutureWarning)


In [2]:
sigmoid = nn.Sigmoid()

class Swish(torch.autograd.Function):
    @staticmethod
    def forward(ctx, i):
        result = i * sigmoid(i)
        ctx.save_for_backward(i)
        return result
    @staticmethod
    def backward(ctx, grad_output):
        i = ctx.saved_variables[0]
        sigmoid_i = sigmoid(i)
        return grad_output * (sigmoid_i * (1 + i * (1 - sigmoid_i)))


class Swish_Module(nn.Module):
    def forward(self, x):
        return Swish.apply(x)

In [13]:
class Effnet_Melanoma(nn.Module):
    def __init__(self, enet_type, out_dim, n_meta_features=0, n_meta_dim=[512, 128], pretrained=False):
        super(Effnet_Melanoma, self).__init__()
        self.n_meta_features = n_meta_features
        self.enet = geffnet.create_model(enet_type, pretrained=pretrained)
        self.dropouts = nn.ModuleList([
            nn.Dropout(0.5) for _ in range(5)
        ])
        in_ch = self.enet.classifier.in_features
        if n_meta_features > 0:
            self.meta = nn.Sequential(
                nn.Linear(n_meta_features, n_meta_dim[0]),
                nn.BatchNorm1d(n_meta_dim[0]),
                Swish_Module(),
                nn.Dropout(p=0.3),
                nn.Linear(n_meta_dim[0], n_meta_dim[1]),
                nn.BatchNorm1d(n_meta_dim[1]),
                Swish_Module(),
            )
            in_ch += n_meta_dim[1]
        self.myfc = nn.Linear(in_ch, out_dim)
        self.enet.classifier = nn.Identity()

    def extract(self, x):
        x = self.enet(x)
        return x

    def forward(self, x, x_meta=None):
        x = self.extract(x).squeeze(-1).squeeze(-1)
        if self.n_meta_features > 0:
            x_meta = self.meta(x_meta)
            x = torch.cat((x, x_meta), dim=1)
        for i, dropout in enumerate(self.dropouts):
            if i == 0:
                out = self.myfc(dropout(x))
            else:
                out += self.myfc(dropout(x))
        out /= len(self.dropouts)
        return out

In [14]:
class args:
    
    kernel_type = "9c_b4ns_448_ext_15ep-newfold"
    data_dir = "./data/"
    data_folder = 512
    image_size = 448
    enet_type = "tf_efficientnet_b4_ns"
    batch_size = 20
    num_workers = 0
    init_lr = 0.0001
    out_dim = 6
    n_epochs = 5
    model_dir = './weights'
    CUDA_VISIBLE_DEVICES = '0'
    fold = '0,1,2,3,4'
    n_meta_dim = '512,128'
    image_folder = "ISIC2019_images"
    
    DEBUG = False
    log_dir = "./logs"
    use_meta = False
    use_amp = False


In [131]:
from sklearn.metrics import classification_report

def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

def train_epoch(model, loader, optimizer):

    model.train()
    train_loss = []
    bar = tqdm(loader)
    for (data, target) in bar:
        
        # print(data.shape)
        optimizer.zero_grad()
        
        if args.use_meta:
            data, meta = data
            data, meta, target = data.to(device), meta.to(device), target.to(device)
            logits = model(data, meta)
        else:
            data, target = data.to(device), target.to(device)
            logits = model(data)        
        
        loss = criterion(logits, target)

        if not args.use_amp:
            loss.backward()
        else:
            with amp.scale_loss(loss, optimizer) as scaled_loss:
                scaled_loss.backward()

        if args.image_size in [896,576]:
            torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)
        optimizer.step()

        loss_np = loss.detach().cpu().numpy()
        train_loss.append(loss_np)
        smooth_loss = sum(train_loss[-100:]) / min(len(train_loss), 100)
        bar.set_description('loss: %.5f, smth: %.5f' % (loss_np, smooth_loss))

    train_loss = np.mean(train_loss)
    return train_loss

def val_epoch(model, loader, mel_idx, is_ext=None, n_test=1, get_output=False):

    model.eval()
    predict = []
    TARGETS = []
    val_loss = []
    with torch.no_grad():
        for (data, target) in loader:
            data, target = data.to(device), target.to(device)
            out = model(data)
            pred = out.argmax(1)
            TARGETS += list(target.cpu().numpy())
            predict += list(pred.cpu().numpy())
            loss = criterion(out, target)
            val_loss.append(loss.detach().cpu().numpy())

        val_loss = np.mean(val_loss)
        acc = (np.array(predict) == np.array(TARGETS)).mean() * 100.
        print(classification_report(TARGETS, predict, target_names = ["MEL", "NV", "BCC", "BKL", "SCC", "AK_DF_VASC"]))
        return val_loss, acc

def run(fold, df, meta_features, n_meta_features, transforms_train, transforms_val, mel_idx):
    
    df_train = df[df["train_fold"] != fold]
    df_valid = df[df["val_fold"] == fold]

    dataset_train = MelanomaDataset(df_train, 'train', meta_features, transform=transforms_train)
    dataset_valid = MelanomaDataset(df_valid, 'valid', meta_features, transform=transforms_val)
    train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=args.batch_size, sampler=RandomSampler(dataset_train), num_workers=0)
    valid_loader = torch.utils.data.DataLoader(dataset_valid, batch_size=args.batch_size, num_workers=0)
    
    if args.enet_type == 'resnest101':
        model = Resnest_Melanoma(
        args.enet_type,
        n_meta_features=n_meta_features,
        n_meta_dim=[int(nd) for nd in args.n_meta_dim.split(',')],
        out_dim=args.out_dim,
        pretrained=True
    )
    elif args.enet_type == 'seresnext101':
        model = Seresnext_Melanoma(
        args.enet_type,
        n_meta_features=n_meta_features,
        n_meta_dim=[int(nd) for nd in args.n_meta_dim.split(',')],
        out_dim=args.out_dim,
        pretrained=True
    )
    elif 'efficientnet' in args.enet_type:
        model = Effnet_Melanoma(
        args.enet_type,
        n_meta_features=n_meta_features,
        n_meta_dim=[int(nd) for nd in args.n_meta_dim.split(',')],
        out_dim=args.out_dim,
        pretrained=True
    )
    else:
        raise NotImplementedError()
    
    model = model.to(device)
    # print(model)

    auc_max = 0.
    model_file  = os.path.join(args.model_dir, f'{args.kernel_type}_best.pth')
    model_file3 = os.path.join(args.model_dir, f'{args.kernel_type}_final.pth')

    optimizer = optim.Adam(model.parameters(), lr=args.init_lr)
    if args.use_amp:
        model, optimizer = amp.initialize(model, optimizer, opt_level="O1")
    if DP:
        model = nn.DataParallel(model)
    scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, args.n_epochs - 1)
    scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)
    
    print(len(dataset_train), len(dataset_valid))

    for epoch in range(1, args.n_epochs + 1):
        print(time.ctime(), f'Epoch {epoch}')

        train_loss = train_epoch(model, train_loader, optimizer)
        val_loss, acc = val_epoch(model, valid_loader, mel_idx, is_ext=df_valid['is_ext'].values)

        content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {train_loss:.5f}, valid loss: {(val_loss):.5f}, acc: {(acc):.4f}.'
        print(content)
        with open(os.path.join(args.log_dir, f'log_{args.kernel_type}.txt'), 'a') as appender:
            appender.write(content + '\n')

        scheduler_warmup.step()    
        if epoch==2: scheduler_warmup.step()
            
    torch.save(model.state_dict(), model_file3)
    

In [121]:
import os
import cv2
import numpy as np
import pandas as pd
import albumentations
import torch
from torch.utils.data import Dataset

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [25]:
class MelanomaDataset(Dataset):
    def __init__(self, csv, mode, meta_features, transform=None):
    
        self.csv = csv.reset_index(drop=True)
        self.target = list(csv["target"])
        self.image = list(csv["filepath"])
        self.mode = mode
        self.use_meta = meta_features is not None
        self.meta_features = meta_features
        self.transform = transform

    def __len__(self):
        return len(self.target)

    def __getitem__(self, index):
        image = np.load(self.image[index][:-4] + ".npy")
        data = torch.tensor(image).float()
#         row = self.csv.iloc[index]
#         image = cv2.imread(row.filepath)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
#         if self.transform is not None:
#             res = self.transform(image=image)
#             image = res['image'].astype(np.float32)
#         else:
#             image = image.astype(np.float32)

#         image = image.transpose(2, 0, 1)

#         if self.use_meta:
#             data = (torch.tensor(image).float(), torch.tensor(self.csv.iloc[index][self.meta_features]).float())
#         else:
#             data = torch.tensor(image).float()

        if self.mode == 'test':
            return data
        else:
            return data, torch.tensor(self.target[index]).long()
        
def get_transforms(image_size):

    transforms_train = albumentations.Compose([
        albumentations.Transpose(p=0.5),
        albumentations.VerticalFlip(p=0.5),
        albumentations.HorizontalFlip(p=0.5),
        albumentations.RandomBrightness(limit=0.2, p=0.75),
        albumentations.RandomContrast(limit=0.2, p=0.75),
        albumentations.OneOf([
            albumentations.MotionBlur(blur_limit=5),
            albumentations.MedianBlur(blur_limit=5),
            albumentations.GaussianBlur(blur_limit=5),
            albumentations.GaussNoise(var_limit=(5.0, 30.0)),
        ], p=0.7),

        albumentations.OneOf([
            albumentations.OpticalDistortion(distort_limit=1.0),
            albumentations.GridDistortion(num_steps=5, distort_limit=1.),
            albumentations.ElasticTransform(alpha=3),
        ], p=0.7),

        albumentations.CLAHE(clip_limit=4.0, p=0.7),
        albumentations.HueSaturationValue(hue_shift_limit=10, sat_shift_limit=20, val_shift_limit=10, p=0.5),
        albumentations.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
        albumentations.Resize(image_size, image_size),
        albumentations.Cutout(max_h_size=int(image_size * 0.375), max_w_size=int(image_size * 0.375), num_holes=1, p=0.7),
        albumentations.Normalize()
    ])

    transforms_val = albumentations.Compose([
        albumentations.Resize(image_size, image_size),
        albumentations.Normalize()
    ])

    return transforms_train, transforms_val

def get_df(kernel_type, out_dim, data_dir, data_folder, use_meta):

    # 2019 csv data 
    df_train2 = pd.read_csv(os.path.join(data_dir, 'train.csv'))
    df_train2 = df_train2.drop("UNK", 1)
    df_train2["AK_DF_VASC"] = df_train2["AK"] + df_train2["DF"] + df_train2["VASC"]
    df_train2 = df_train2.drop(["AK", "DF","VASC"], 1)
    
    # index - 3, 5, 6
    # AK, DF, VASC
    diagnosis2idx = {}
    df_train2.columns[1:]
    for i in df_train2.columns[1:]:
        diagnosis2idx[i] = len(diagnosis2idx)
    idx2diagnosis = {v:k for k, v in diagnosis2idx.items()}

    idx = np.argmax(np.array(df_train2)[:, 1:], axis = 1)
    df_train2['target']  = idx
    df_train2['diagnosis'] = df_train2['target'].map(idx2diagnosis)
    df_train2['filepath'] = df_train2['image'].apply(lambda x: os.path.join("data", args.image_folder, f'{x}.jpg'))
    
    df_train2['tfrecord'] = list(range(len(idx)))
    df_train2['train_fold'] = df_train2['tfrecord'] % 8
    df_train2['val_fold'] = df_train2['tfrecord'] % 6
    df_train2['is_ext'] = 1
    
    types = list(diagnosis2idx.keys())
    samples = df_train2[df_train2[types[0]]==1].sample(600)
    for tp in types[1:]:
        tem = df_train2[df_train2[tp]==1].sample(600)
        samples = samples.append(tem)
    samples = samples.sort_values("image")
    
    df_test = samples.sample(100)
    meta_features = None
    n_meta_features = 0

    # class mapping
    mel_idx = diagnosis2idx['MEL']

    return samples, df_test, meta_features, n_meta_features, mel_idx, diagnosis2idx


In [18]:
from warmup_scheduler import GradualWarmupScheduler 

class GradualWarmupSchedulerV2(GradualWarmupScheduler):
    def __init__(self, optimizer, multiplier, total_epoch, after_scheduler=None):
        super(GradualWarmupSchedulerV2, self).__init__(optimizer, multiplier, total_epoch, after_scheduler)
    def get_lr(self):
        if self.last_epoch > self.total_epoch:
            if self.after_scheduler:
                if not self.finished:
                    self.after_scheduler.base_lrs = [base_lr * self.multiplier for base_lr in self.base_lrs]
                    self.finished = True
                return self.after_scheduler.get_lr()
            return [base_lr * self.multiplier for base_lr in self.base_lrs]
        if self.multiplier == 1.0:
            return [base_lr * (float(self.last_epoch) / self.total_epoch) for base_lr in self.base_lrs]
        else:
            return [base_lr * ((self.multiplier - 1.) * self.last_epoch / self.total_epoch + 1.) for base_lr in self.base_lrs]


In [19]:
def set_seed(seed=0):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True

In [20]:
os.makedirs(args.model_dir, exist_ok=True)
os.makedirs(args.log_dir, exist_ok=True)
os.environ['CUDA_VISIBLE_DEVICES'] = args.CUDA_VISIBLE_DEVICES

DP = len(os.environ['CUDA_VISIBLE_DEVICES']) > 1

set_seed()

device = torch.device('cuda')
criterion = nn.CrossEntropyLoss()

In [39]:
# use transfer learning for ISIC 2019    
df, df_test, meta_features, n_meta_features, mel_idx, diagnosis2idx = get_df(
    args.kernel_type,
    args.out_dim,
    args.data_dir,
    args.data_folder,
    args.use_meta
)

transforms_train, transforms_val = get_transforms(args.image_size)
fold = 0
run(fold, df, meta_features, n_meta_features, transforms_train, transforms_val, mel_idx)

In [22]:
# 运行配置 
main()

  0%|                                                                                          | 0/158 [00:00<?, ?it/s]

3145 628
Wed May 18 13:22:53 2022 Epoch 1


  0%|                                                                                          | 0/158 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         MEL       0.51      0.48      0.50        95
          NV       0.59      0.84      0.69       113
         BCC       0.72      0.46      0.56       120
         BKL       0.62      0.51      0.56       107
         SCC       0.48      0.64      0.55        94
  AK_DF_VASC       0.55      0.48      0.52        99

    accuracy                           0.57       628
   macro avg       0.58      0.57      0.56       628
weighted avg       0.59      0.57      0.57       628

Wed May 18 13:24:11 2022 Epoch 1, lr: 0.0001000, train loss: 1.54042, valid loss: 1.14749, acc: 57.1656.
Wed May 18 13:24:11 2022 Epoch 2


  0%|                                                                                          | 0/158 [00:00<?, ?it/s]

...

              precision    recall  f1-score   support

         MEL       0.52      0.36      0.42        95
          NV       0.64      0.86      0.73       113
         BCC       0.74      0.65      0.69       120
         BKL       0.66      0.55      0.60       107
         SCC       0.63      0.55      0.59        94
  AK_DF_VASC       0.54      0.74      0.63        99

    accuracy                           0.63       628
   macro avg       0.62      0.62      0.61       628
weighted avg       0.63      0.63      0.62       628

Wed May 18 13:26:38 2022 Epoch 3, lr: 0.0008536, train loss: 1.17538, valid loss: 1.03590, acc: 62.5796.
Wed May 18 13:26:38 2022 Epoch 4


  0%|                                                                                          | 0/158 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         MEL       0.83      0.71      0.76        95
          NV       0.81      0.90      0.85       113
         BCC       0.79      0.89      0.84       120
         BKL       0.85      0.76      0.80       107
         SCC       0.75      0.78      0.76        94
  AK_DF_VASC       0.80      0.76      0.78        99

    accuracy                           0.80       628
   macro avg       0.81      0.80      0.80       628
weighted avg       0.81      0.80      0.80       628

Wed May 18 13:27:53 2022 Epoch 4, lr: 0.0005000, train loss: 0.75688, valid loss: 0.66143, acc: 80.4140.
Wed May 18 13:27:53 2022 Epoch 5


loss: 0.27291, smth: 0.27130: 100%|██████████████████████████████████████████████████| 158/158 [01:08<00:00,  2.29it/s]


              precision    recall  f1-score   support

         MEL       0.91      0.81      0.86        95
          NV       0.85      0.94      0.89       113
         BCC       0.89      0.90      0.90       120
         BKL       0.90      0.86      0.88       107
         SCC       0.82      0.82      0.82        94
  AK_DF_VASC       0.91      0.93      0.92        99

    accuracy                           0.88       628
   macro avg       0.88      0.88      0.88       628
weighted avg       0.88      0.88      0.88       628

Wed May 18 13:29:06 2022 Epoch 5, lr: 0.0001464, train loss: 0.30915, valid loss: 0.45456, acc: 87.8981.


## Third Part Dataset

In [133]:
pretrained_ISIC = Effnet_Melanoma(
        args.enet_type,
        n_meta_features=n_meta_features,
        n_meta_dim=[int(nd) for nd in args.n_meta_dim.split(',')],
        out_dim=args.out_dim,
        pretrained=True
    )

pretrained_ISIC.load_state_dict(
    torch.load("weights/9c_b4ns_448_ext_15ep-newfold_final.pth"))

<All keys matched successfully>

In [134]:
def readDir(dirPath):
    allFiles = []
    if os.path.isdir(dirPath):
        fileList = os.listdir(dirPath)
        for f in fileList:
            f = dirPath+'/'+f
            if os.path.isdir(f):
                subFiles = readDir(f)
                allFiles = subFiles + allFiles
            elif f.endswith("jpg"):
#                 if "微信截图" in f:
#                     f2 = f.replace("微信截图_", "")
#                     os.rename(f, f2)
#                     allFiles.append(f2)
#                 else:
                allFiles.append(f)
        return allFiles
    else:
        return 'Error,not a dir'

In [135]:
images = readDir("OurData")

In [136]:
labels = []
for path in images:
    class_ = path.split("/")[1]
    if "df" in class_:
        labels.append(5)
    else:
        labels.append(diagnosis2idx[class_.upper()])

In [137]:
split = 10
train_split = list(range(split)) * 200
train_split = np.array(train_split[:len(images)])
val_split = list(range(int(split/2))) * 200
val_split = np.array(val_split[:len(images)])

images = np.array(images)
labels = np.array(labels)

In [127]:
# transfer the data into images
for path in tqdm(images):
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    res = transforms_train(image=image)
    image = res['image'].astype(np.float32)
    image = image.transpose(2, 0, 1)
    # print(image.shape)
    np.save(path[:-4]+".npy", image)

100%|████████████████████████████████████████████████████████████████████████████████| 652/652 [00:59<00:00, 11.04it/s]


In [138]:
# images, labels, - train_split, val_split
class ourDataset(Dataset):
    def __init__(self, image, labels, transform=None):
    
        self.target = labels
        self.image = image
        self.transform = transform

    def __len__(self):
        return len(self.target)

    def __getitem__(self, index):
        image = np.load(self.image[index][:-4] + ".npy")
        data = torch.tensor(image).float()
        return data, torch.tensor(self.target[index]).long()

In [139]:
flag = 0
train_image = images[train_split!=flag]
val_image = images[val_split==flag]
train_label = labels[train_split!=flag]
val_label = labels[val_split==flag]

train_data = ourDataset(train_image, train_label)
val_data = ourDataset(val_image, val_label)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=args.batch_size, sampler=RandomSampler(train_data), num_workers=0)
valid_loader = torch.utils.data.DataLoader(val_data, batch_size=args.batch_size, num_workers=0)

In [140]:
model = pretrained_ISIC.to(device)
al_loss, acc = val_epoch(model, valid_loader, mel_idx)

              precision    recall  f1-score   support

         MEL       0.47      0.50      0.48        14
          NV       0.23      0.25      0.24        20
         BCC       0.59      0.21      0.31        47
         BKL       0.06      0.50      0.10         4
         SCC       0.11      0.08      0.10        12
  AK_DF_VASC       0.44      0.41      0.42        34

    accuracy                           0.30       131
   macro avg       0.31      0.33      0.28       131
weighted avg       0.42      0.30      0.32       131



In [132]:
# print(model)
auc_max = 0.

model_file3 = os.path.join(args.model_dir, 'third_part_final.pth')

optimizer = optim.Adam(model.parameters(), lr=args.init_lr)
if args.use_amp:
    model, optimizer = amp.initialize(model, optimizer, opt_level="O1")
if DP:
    model = nn.DataParallel(model)
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, args.n_epochs - 1)
scheduler_warmup = GradualWarmupSchedulerV2(optimizer, multiplier=10, total_epoch=1, after_scheduler=scheduler_cosine)

print(len(train_data), len(val_data))

for epoch in range(1, args.n_epochs + 1):
    print(time.ctime(), f'Epoch {epoch}')

    train_loss = train_epoch(model, train_loader, optimizer)
    val_loss, acc = val_epoch(model, valid_loader, mel_idx)

    content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {train_loss:.5f}, valid loss: {(val_loss):.5f}, acc: {(acc):.4f}.'
    print(content)
    with open(os.path.join(args.log_dir, f'log_{args.kernel_type}.txt'), 'a') as appender:
        appender.write(content + '\n')

    scheduler_warmup.step()    
    if epoch==2: scheduler_warmup.step()

torch.save(model.state_dict(), model_file3)

  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

586 131
Wed May 18 15:15:36 2022 Epoch 1


  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         MEL       0.91      0.71      0.80        14
          NV       0.77      0.85      0.81        20
         BCC       0.79      0.72      0.76        47
         BKL       0.40      0.50      0.44         4
         SCC       0.67      0.33      0.44        12
  AK_DF_VASC       0.64      0.82      0.72        34

    accuracy                           0.73       131
   macro avg       0.70      0.66      0.66       131
weighted avg       0.74      0.73      0.72       131

Wed May 18 15:15:49 2022 Epoch 1, lr: 0.0001000, train loss: 1.30119, valid loss: 0.79397, acc: 72.5191.
Wed May 18 15:15:49 2022 Epoch 2


  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

...

              precision    recall  f1-score   support

         MEL       0.91      0.71      0.80        14
          NV       0.89      0.80      0.84        20
         BCC       0.77      0.94      0.85        47
         BKL       0.75      0.75      0.75         4
         SCC       0.78      0.58      0.67        12
  AK_DF_VASC       0.88      0.82      0.85        34

    accuracy                           0.82       131
   macro avg       0.83      0.77      0.79       131
weighted avg       0.83      0.82      0.82       131

Wed May 18 15:16:15 2022 Epoch 3, lr: 0.0008536, train loss: 0.17371, valid loss: 0.70005, acc: 82.4427.
Wed May 18 15:16:15 2022 Epoch 4


  0%|                                                                                           | 0/30 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         MEL       0.86      0.86      0.86        14
          NV       0.89      0.85      0.87        20
         BCC       0.89      0.87      0.88        47
         BKL       0.80      1.00      0.89         4
         SCC       1.00      0.50      0.67        12
  AK_DF_VASC       0.80      0.97      0.88        34

    accuracy                           0.86       131
   macro avg       0.87      0.84      0.84       131
weighted avg       0.87      0.86      0.86       131

Wed May 18 15:16:28 2022 Epoch 4, lr: 0.0005000, train loss: 0.09414, valid loss: 0.53898, acc: 86.2595.
Wed May 18 15:16:28 2022 Epoch 5


loss: 0.29429, smth: 0.05201: 100%|████████████████████████████████████████████████████| 30/30 [00:12<00:00,  2.49it/s]


              precision    recall  f1-score   support

         MEL       0.86      0.86      0.86        14
          NV       0.89      0.80      0.84        20
         BCC       0.86      0.89      0.88        47
         BKL       0.80      1.00      0.89         4
         SCC       1.00      0.58      0.74        12
  AK_DF_VASC       0.84      0.94      0.89        34

    accuracy                           0.86       131
   macro avg       0.87      0.85      0.85       131
weighted avg       0.87      0.86      0.86       131

Wed May 18 15:16:41 2022 Epoch 5, lr: 0.0001464, train loss: 0.05201, valid loss: 0.54129, acc: 86.2595.
